# How magnetic waves are used to construct non-invasive internal anatomic and physiologic images 


## Let's start with how the way we hear sounds is similar to how we image with MRI:

When plucking a string, one can hear a note. That note comes from oscillation at a particular frequency. Notice one note has one frequency, while two notes will have two frequencies, and three notes will have three frequencies. This can be shown with the guitar! 

Imagine how useful it would be if instead of seeing just the sum of those sinusoids, instead we could get the frequency, amplitude, and phase of the sinusoids that are making up that note!

This is called a "fourier transform". This means that every single signal can be decomposed into the sum of sinusoids of varying frequencies. In the fourier domain, it is represented by a delta function at the frequency of the wave, with the amplitude and phase saved as the height of the delta function and the complex portion of the signal. 

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import pickle
import pydicom as dicom
import matplotlib.image as mpimg

%matplotlib

ModuleNotFoundError: No module named 'pydicom'

# Here's an example with a sound wave of 5Hz and a sound wave of 7Hz: 

Starting with one sinusoid, we can see the fourier transform of it in the fourier domain and how it shifts location as the frequency increases! 

Then, we can see how the combination of the two can be shown as the sum of two sinusoids in fourier space!

In [6]:

img = mpimg.imread('data/Fig1.png')
pl.imshow(img)
pl.show()

# Now how can we see this in an image? 

Let's make those waves two dimensional! They're planar waves now, 2D! 

Of course, this means they will be two frequency space? 

That doesn't make any sense intuitively, does it... so let's take a look!

Here you see that the 2D planar wave of a single frequency is again actually just two delta functions in a 2D plane! 

So what does their location in the 2D space mean? 

In [105]:
img = mpimg.imread('data/Fig2.png')
pl.imshow(img)
pl.show()

# What does 2D wave mean?

A 1D wave doesn't have any direction, it only has frequency. (this can be hard to understand, but if you think about it, even for a 1D wave to travel in a direction, that direction has to be a direction in at least 2D space!) 

But a 2D wave has both a frequency AND a direction in which it is going! This means that the direction of the frequency is also encoded in its location in the 2D frequency space.

As we rotate the planar waves to have them propagating in different directions, we can see the delta functions also rotating in k-space!

Below we can see that the location of the delta waves in the 2D space is the direction of the wave! One 2D wave (5Hz) is rotated about the center, and we can see that the delta waves in 2D space rotate with it.

In [119]:
with open('data/Fig3_Rotating.pickle','wb') as handle: 
    [image_rotating,kspace_rotating] = pickle.load(handle)

# What about the 2D frequency though? 

How is the frequency of the wave encoded then? 

Remember the 1D wave? The 5Hz wave was 5 away from the center, while the 7Hz wave was 7 away from the center. In 2D space then, a 2D 5Hz wave will be 5 from the 2D origin (0,0) while aa 7Hz wave will be 7 away from the 2D origin! 

This means that the frequency is from the distance of the delta function from the center, while the direction of the wave is the angle of the delta function from the center! 

Below is an example of a wave that is increasing in frequency while rotating. We can see that the fourier transform is showing the delta waves are moving away from the center and rotating as well. 

In [120]:
fig, (ax1, ax2) = pl.subplots(1,2)
angles = np.linspace(0,np.pi,20)
wavelengths = np.linspace(20,3,20)
ax1.set_title('Planar Wave')
ax2.set_title('FT of Planar Wave')

kspace_rotateincrease = np.zeros((len(img[0]),len(img[0]),20))
image_rotateincrease =np.zeros((len(img[0]),len(img[0]),20))
count = 0
for j in range(len(angles)):
    #phi being the tilt angle
    img = np.sin(2*np.pi*(X*np.cos(angles[j]) + Y*np.sin(angles[j])) / wavelengths[j])
    ax1.imshow(img,cmap = 'gray')
    ft = np.fft.fft2(img)
    kspace = np.fft.fftshift(ft) #moves zero frequency component to center
    ax2.imshow(np.abs(kspace),cmap = 'gray')

    image_rotateincrease[:,:,count]
    kspace_rotateincrease[:,:,count]
    count = count+1
    pl.pause(.5)
    

with open('data/Fig3_Rotating.pickle','wb') as handle: 
    pickle.dump([image_rotateincrease,kspace_rotateincrease],handle)

In [33]:
name = 'Coronal' #Sagittal, Axial
ds = dicom.dcmread('data/' + name + '.dcm')
img = ds.pixel_array

fov_img = 256

with open('data/'+name + '_superposition.pickle','rb') as handle1:
    stack_image = pickle.load(handle1)
with open('data/'+ name +'_superposition_masks.pickle','rb') as handle2:
    stack_waves = pickle.load(handle2)
with open('data/'+ name + '_superposition_kspace.pickle','rb') as handle3:
    stack_kspace = pickle.load(handle3)
    
fig, (ax1, ax2, ax3,ax4) = pl.subplots(1,4)
ax4.imshow(img,cmap = 'gray')
ax4.set_title('Final Image')
for i in np.arange(0,np.shape(stack_image)[2]):
    ax1.imshow(stack_kspace[:,:,i],vmin = 0,vmax = 6000,extent = [-.5, .5, -.5, .5],cmap = 'gray')
    ax1.set_title('k-space')
    ax2.imshow(stack_waves[:,:,i],cmap = 'gray')
    ax2.set_title('planar\nwaves')
    ax3.imshow(stack_image[:,:,i],cmap = 'gray')
    ax3.set_title('Image\nForming')
    pl.pause(.1)